In [1]:
import lbcred, os
from lbcred import image, interactive, tools, reduce

In [10]:
default_config = os.path.join(lbcred.project_dir, 'lbcreduce-config.yml')
initial_config = tools.initialize_config(default_config)
config, dir_overwritten = interactive.initialize_directories(initial_config)
image_info = image.get_image_info(config)
image.check_files(image_info, config)
image_info = image.separate_chips(image_info, config)
image_info

filename,imagetyp,object,propid,filter,instrument
str31,str6,str9,str13,str8,str8
lbcr.20191221.134032-chip2.fits,flat,SkyFlat,flatcalib,R-BESSEL,LBC-RED
lbcr.20191221.034630-chip2.fits,object,NGC628LSB,OSU_dwarfsimg,R-BESSEL,LBC-RED
lbcb.20191220.004744-chip2.fits,flat,SkyFlat,flatcalib,B-BESSEL,LBC_BLUE
lbcr.20191221.035330-chip2.fits,object,NGC628LSB,OSU_dwarfsimg,R-BESSEL,LBC-RED
lbcb.20191221.132846-chip2.fits,flat,SkyFlat,flatcalib,B-BESSEL,LBC_BLUE
lbcr.20191221.133650-chip2.fits,flat,SkyFlat,flatcalib,R-BESSEL,LBC-RED
lbcb.20191220.004910-chip2.fits,flat,SkyFlat,flatcalib,B-BESSEL,LBC_BLUE
lbcr.20191220.004741-chip2.fits,flat,SkyFlat,flatcalib,R-BESSEL,LBC-RED
lbcb.20191221.133234-chip2.fits,flat,SkyFlat,flatcalib,B-BESSEL,LBC_BLUE


In [ ]:
tools.plot_overscan()